# Install and import libraries

In [65]:
%pip install pydot
%pip install tensorflow
%pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,
    METADATA_PATH,
)

from constants import CLASSES
num_classes = len(CLASSES)

# Read the files in the data dir

## Convert .csv(s) to dataframes and concatenate

In [ ]:
from pathlib import Path
import pandas as pd

dfs = []

# Set your input path
DATA_INPUT_PATH = Path(DATA_INPUT_PATH)  # ← safer path format

# Find CSV files
files = list(DATA_INPUT_PATH.glob("*.csv"))
print("Looking in:", DATA_INPUT_PATH)
print("Found CSVs:", files)

if not files:
    raise FileNotFoundError(f"No CSV files found in {DATA_INPUT_PATH.resolve()}")

# Read all files
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate into one DataFrame
df = pd.concat(dfs, axis=0)

# Reorder/select columns (if applicable)

# Remove duplicates
print(f"Shape before dropping duplicates: {df.shape}")
df = df.drop_duplicates()
print(f"Shape after dropping duplicates: {df.shape}")
df.columns = df.columns.str.strip()

Looking in: C:\Users\blacb\Documents\GitHub\NeuroSyn\model-v4\dataTemp
Found CSVs: [WindowsPath('C:/Users/blacb/Documents/GitHub/NeuroSyn/model-v4/dataTemp/physio_emg_imu_data_20250430_002614.csv')]
Shape before dropping duplicates: (10523, 88)
Shape after dropping duplicates: (10523, 88)


In [69]:
import pandas as pd
import numpy as np
# from constants import NUM_EMG_SENSORS # Make sure NUM_EMG_SENSORS is defined (should be 8)

# Assuming you load your data like this:
# df = pd.read_csv('your_file.csv')

# --- Add this line to clean up column names ---
df.columns = df.columns.str.strip()
# ----------------------------------------------

# Debug: Print all column names to verify
print("DEBUG: All column names after cleanup:")
print(list(df.columns))

NUM_EMG_SENSORS = 8

print("Calculating calibration statistics from 'Rest' data...")

# Select only the 'Rest' data (gesture_id == 0)
rest_df = df[df['gesture_id'] == 0]

# Select EMG columns (s1 to s8)
emg_columns = [f"s{i}" for i in range(1, NUM_EMG_SENSORS + 1)]

# Select IMU columns
imu_columns = ["quat_w", "quatx", "quaty", "quatz"]

# Define landmark column names
landmark_column_names = [c for c in df.columns if c.endswith(("_x", "_y", "_z"))]

# Debug: Print detected landmark columns
print(f"Landmark columns: {landmark_column_names}")
print(f"DEBUG: Found {len(landmark_column_names)} landmark column names.")

# Check if the expected number of landmark columns (72) is found
if len(landmark_column_names) != 72:
    print("WARNING: Expected 72 landmark columns, but found", len(landmark_column_names))
    print("DEBUG: Found columns:", landmark_column_names)
    # Optionally raise an error
    # raise ValueError(f"Expected 72 landmark columns, found {len(landmark_column_names)}")

# Ensure rest data exists before calculating stats
if 'gesture_id' in df.columns and 0 in df['gesture_id'].values:
    rest_df = df[df['gesture_id'] == 0].copy()
    rest_emg_data = rest_df[emg_columns].values

    if len(rest_emg_data) > 0:
        calibration_mean = np.mean(rest_emg_data, axis=0)
        calibration_std = np.std(rest_emg_data, axis=0)
        calibration_std[calibration_std < 1e-6] = 1.0

        print(f"Calculated Calibration Mean (shape {calibration_mean.shape}):\n{np.round(calibration_mean, 2)}")
        print(f"Calculated Calibration StdDev (shape {calibration_std.shape}):\n{np.round(calibration_std, 2)}")
    else:
        print("ERROR: No 'Rest' data rows found (gesture_id == 0)!")
        calibration_mean = np.zeros(NUM_EMG_SENSORS)
        calibration_std = np.ones(NUM_EMG_SENSORS)
        print("Using default calibration stats (mean=0, std=1). CHECK YOUR DATA.")
else:
    print("ERROR: 'gesture_id' column missing or no 'Rest' data found.")
    calibration_mean = np.zeros(NUM_EMG_SENSORS)
    calibration_std = np.ones(NUM_EMG_SENSORS)
    print("Using default calibration stats (mean=0, std=1). CHECK YOUR DATA.")

DEBUG: All column names after cleanup:
['id', 'time', 'gesture_id', 'gesture_name', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 'quat_w', 'quatx', 'quaty', 'quatz', 'Wrist_x', 'Wrist_y', 'Wrist_z', 'Thumb_CMC_x', 'Thumb_CMC_y', 'Thumb_CMC_z', 'Thumb_MCP_x', 'Thumb_MCP_y', 'Thumb_MCP_z', 'Thumb_IP_x', 'Thumb_IP_y', 'Thumb_IP_z', 'Thumb_Tip_x', 'Thumb_Tip_y', 'Thumb_Tip_z', 'Index_MCP_x', 'Index_MCP_y', 'Index_MCP_z', 'Index_PIP_x', 'Index_PIP_y', 'Index_PIP_z', 'Index_DIP_x', 'Index_DIP_y', 'Index_DIP_z', 'Index_Tip_x', 'Index_Tip_y', 'Index_Tip_z', 'Middle_MCP_x', 'Middle_MCP_y', 'Middle_MCP_z', 'Middle_PIP_x', 'Middle_PIP_y', 'Middle_PIP_z', 'Middle_DIP_x', 'Middle_DIP_y', 'Middle_DIP_z', 'Middle_Tip_x', 'Middle_Tip_y', 'Middle_Tip_z', 'Ring_MCP_x', 'Ring_MCP_y', 'Ring_MCP_z', 'Ring_PIP_x', 'Ring_PIP_y', 'Ring_PIP_z', 'Ring_DIP_x', 'Ring_DIP_y', 'Ring_DIP_z', 'Ring_Tip_x', 'Ring_Tip_y', 'Ring_Tip_z', 'Pinky_MCP_x', 'Pinky_MCP_y', 'Pinky_MCP_z', 'Pinky_PIP_x', 'Pinky_PIP_y', 'Pinky

In [70]:
# --- MODIFIED CELL 5: Sliding-window Feature Extraction with Z-Score ---
import numpy as np
from constants import WINDOW_SIZE, WINDOW_STEP, NUM_EMG_SENSORS # Added NUM_EMG_SENSORS

# CELL 2 (Corrected)
# --- MODIFIED CELL: Sliding-window Feature Extraction & Target Creation ---
import numpy as np
# Assuming constants WINDOW_SIZE, WINDOW_STEP, NUM_EMG_SENSORS are defined
# Assuming calibration_mean, calibration_std from CELL 1 exist
# Assuming landmark_column_names from CELL 1 exists
# Assuming df is your loaded DataFrame

# define a real scaler object
scaler = StandardScaler()
scaler.mean_, scaler.scale_ = calibration_mean, calibration_std

print("Starting feature extraction and landmark target creation...")

# Select EMG+IMU columns for features
feature_channels = [f"s{i}" for i in range(1, NUM_EMG_SENSORS + 1)] + ["quat_w", "quatx", "quaty", "quatz"]
raw_features = df[feature_channels].values      # shape = [total_samples, 12]

# Select landmark columns for targets using the list from CELL 1
# Check if landmark_column_names actually exists and is not empty
if 'landmark_column_names' not in globals() or not landmark_column_names:
     raise NameError("landmark_column_names is not defined or empty. Ensure CELL 1 ran correctly and found columns.")
if len(landmark_column_names) != 72:
     print(f"WARNING: Proceeding with {len(landmark_column_names)} landmark columns, but model expects 72. Error is likely.")
     # Or raise ValueError("Incorrect number of landmark columns found.")



raw_landmarks = df[landmark_column_names].values # shape = [total_samples, num_landmark_columns (hopefully 72)]

print(f"names: {df[landmark_column_names]}")

X, Y = [], [] # Uppercase Y for landmark targets
num_windows = 0

for start in range(0, len(raw_features) - WINDOW_SIZE + 1, WINDOW_STEP):
    num_windows += 1
    window_features = raw_features[start : start + WINDOW_SIZE] # shape = [WINDOW_SIZE, 12]

    # Separate EMG and IMU parts of the window
    emg_part = window_features[:, :NUM_EMG_SENSORS]   # shape = [WINDOW_SIZE, 8]
    imu_part = window_features[:, NUM_EMG_SENSORS:]   # shape = [WINDOW_SIZE, 4]

    # Apply Z-score normalization to EMG part
    try:
        normalized_emg_part = (emg_part - calibration_mean) / calibration_std
    except NameError:
        print("ERROR: calibration_mean or calibration_std not defined. Make sure CELL 1 was run.")
        break # Stop processing

    # Calculate RMS on NORMALIZED EMG and ORIGINAL IMU
    rms_emg = np.sqrt(np.mean(normalized_emg_part**2, axis=0))
    rms_imu = np.sqrt(np.mean(imu_part**2, axis=0))

    # Concatenate features
    rms_feat = np.concatenate([rms_emg, rms_imu])
    X.append(rms_feat)

    # Assign the window’s “center” LANDMARK data as the target
    center_idx = start + WINDOW_SIZE // 2
    Y.append(raw_landmarks[center_idx]) # <-- Append the landmark row (should have 72 values)

X = np.array(X)  # shape = [n_windows, 12]
Y = np.array(Y)  # shape = [n_windows, 72] <-- Should now have 72 columns

print(f"Processed {num_windows} windows.")
print("X shape (RMS features):", X.shape)
print("Y shape (Landmark targets):", Y.shape) # <-- Verify this shows (..., 72)

# Check Y shape explicitly
if Y.shape[1] != 72:
    print(f"ERROR: Y shape is {Y.shape}, expected second dimension to be 72.")
    print("       Check the landmark columns in your CSV and CELL 1.")

# --- END OF CORRECTED CELL 2 ---

Starting feature extraction and landmark target creation...
names:         Wrist_x   Wrist_y       Wrist_z  Thumb_CMC_x  Thumb_CMC_y  \
0      0.501415  0.497794  2.854314e-07     0.448627     0.461873   
1      0.499809  0.495536  2.839110e-07     0.448270     0.460057   
2      0.501324  0.502592  2.867195e-07     0.448728     0.465319   
3      0.501258  0.506695  2.854746e-07     0.450308     0.469425   
4      0.501871  0.506546  2.945126e-07     0.450076     0.467540   
...         ...       ...           ...          ...          ...   
10518  0.502680  0.503380  2.788704e-07     0.451335     0.465506   
10519  0.502589  0.502521  2.797877e-07     0.451682     0.464579   
10520  0.503018  0.504544  2.750186e-07     0.451561     0.465724   
10521  0.503794  0.503255  2.858833e-07     0.452157     0.465333   
10522  0.505278  0.504900  2.815112e-07     0.453410     0.466702   

       Thumb_CMC_z  Thumb_MCP_x  Thumb_MCP_y  Thumb_MCP_z  Thumb_IP_x  ...  \
0        -0.013308     0.4

## Scale, split, and one-shot encode RMS feature windows

In [71]:
# CELL 3 (Should now work correctly if Y from CELL 2 is shape [?, 72])
from sklearn.model_selection import train_test_split

# 1) train+val vs test
X_trainval, X_test, Y_trainval, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42 # Using X and Y from Corrected CELL 2
)
# 2) train vs val
X_train, X_val, Y_train, Y_val = train_test_split(
    X_trainval, Y_trainval, test_size=0.25, random_state=42
)

print("Train:", X_train.shape, Y_train.shape) # Y_train shape should be [?, 72]
print("Val:  ", X_val.shape,   Y_val.shape)  # Y_val shape should be [?, 72]
print("Test: ", X_test.shape,  Y_test.shape) # Y_test shape should be [?, 72]

Train: (252, 12) (252, 72)
Val:   (84, 12) (84, 72)
Test:  (84, 12) (84, 72)


## Build, compile & train a simple dense classifier

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(72, activation="linear") # 72 = 63 (hands) + 9 (shoulder elbow wrist)
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model.compile(
    optimizer=Adam(1e-3),
    loss='mse',
    metrics=['mae']
)

# fit with explicit validation_data
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=100,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3, restore_best_weights=True
    )]
)

test_mse, test_mae = model.evaluate(X_test, Y_test)
print(f"Final test MSE: {test_mse:.4f}, MAE: {test_mae:.4f}")

Epoch 1/100


c:\Users\blacb\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.6793 - mae: 0.6150 - val_loss: 2.3702 - val_mae: 0.9842
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5421 - mae: 0.5523 - val_loss: 0.9929 - val_mae: 0.6668
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.4576 - mae: 0.5079 - val_loss: 0.5276 - val_mae: 0.5084
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3486 - mae: 0.4454 - val_loss: 0.3378 - val_mae: 0.4219
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3036 - mae: 0.4222 - val_loss: 0.2527 - val_mae: 0.3736
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2908 - mae: 0.4090 - val_loss: 0.2048 - val_mae: 0.3424
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.2443 - mae: 0.3753 - val_loss: 0.1756 - val_mae: 0.3210
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2239 - mae: 0.3587 - val_loss: 0.1552 - val_mae: 0.3040
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2019 - mae: 0.3418 - val

## Save the model and the scalar + feature list

In [ ]:
# --- MODIFIED CELL 8: Save Model and Metadata ---
# make sure the folder exists
(Path(MODEL_PATH).parent).mkdir(parents=True, exist_ok=True)

# Save the trained model (this line is unchanged)

# CHANGED TO TEMP PATH, REPLACE WITH MODEL_PATH LATER
model.save(MODEL_PATH)
print("Saved model to", MODEL_PATH)

# pickle the scaler, feature names, AND calibration stats

with open(MODEL_PATH, "wb") as f:
    # channels list we used for X is:
    feature_names = [f"s{i}" for i in range(1, NUM_EMG_SENSORS + 1)] + ["quat_w", "quat_x", "quat_y", "quat_z"]
    # <<< Save all three items in a tuple >>>
    try:
        pickle.dump((scaler, feature_names, calibration_mean, calibration_std), f)

        # REPLACE WITH METADATA_PATH
        print("Saved metadata (scaler, features, calib_mean, calib_std) to", METADATA_PATH)
    except NameError:
        print("ERROR: Could not save metadata. Was calibration_mean/std calculated earlier?")
    except Exception as e:
        print(f"ERROR saving metadata: {e}")


# --- END OF MODIFIED CELL 8 ---

Saved model to model\physio_model.h5
Saved metadata (scaler, features, calib_mean, calib_std) to model-v4\model_temp\physio_temp_meta.pkl


In [74]:
import os
from constants import DATA_INPUT_PATH, MODEL_INPUT_PATH, MODEL_PATH, METADATA_PATH

print("DATA_INPUT_PATH ->", DATA_INPUT_PATH)
print("  contains:", os.listdir(DATA_INPUT_PATH))
print()
print("MODEL_INPUT_PATH ->", MODEL_INPUT_PATH)
print("  contains:", os.listdir(MODEL_INPUT_PATH))
print()
print("MODEL_PATH       ->", MODEL_PATH, "exists?", os.path.exists(MODEL_PATH))
print("METADATA_PATH    ->", METADATA_PATH, "exists?", os.path.exists(METADATA_PATH))


DATA_INPUT_PATH -> data
  contains: ['artemis_archive', 'convert_old_data.py', 'physio_emg_imu_data_20250429_001144.csv', 'physio_emg_imu_data_20250429_174128.csv', 'physio_emg_imu_data_20250429_175521.csv', 'physio_emg_imu_data_20250430_002614.csv']

MODEL_INPUT_PATH -> model
  contains: ['artemis_archive', 'physio_metadata.pkl', 'physio_model.h5']

MODEL_PATH       -> model\physio_model.h5 exists? True
METADATA_PATH    -> model\physio_metadata.pkl exists? True


In [75]:
import constants
print(constants.__file__)


c:\Users\blacb\Documents\GitHub\NeuroSyn\model-v4\constants.py


In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from tensorflow.keras.models import load_model
import pickle

from constants import DATA_INPUT_PATH, MODEL_PATH, METADATA_PATH, CLASSES, WINDOW_SIZE

# — 1) load the freshest CSV —
data_dir = Path(DATA_INPUT_PATH)
csv_path = max(data_dir.glob("*.csv"), key=lambda p: p.stat().st_mtime)
df      = pd.read_csv(csv_path)

# — 2) load scaler + feature names —
with open(METADATA_PATH, "rb") as f:
    scaler, feature_names, calibration_mean, calibration_std = pickle.load(f)

# — 3) load your trained model —
model = load_model(MODEL_PATH)

# — 4) for each class, grab the first 100 consecutive rows of that gesture,
#         compute the 12-dim RMS feature vector, and predict —
demo_rows = []
for gid, name in CLASSES.items():
    sub = df[df["gesture_id"] == gid]
    if len(sub) < WINDOW_SIZE:
        print(f"⚠️  not enough samples for '{name}' (need {WINDOW_SIZE}, got {len(sub)})")
        continue
    
    window  = sub[feature_names].values[:WINDOW_SIZE]
    # Separate EMG/IMU parts from the 'window' variable
    emg_part = window[:, :NUM_EMG_SENSORS]
    imu_part = window[:, NUM_EMG_SENSORS:]

    # Apply Z-score normalization to EMG part using LOADED stats
    # (Make sure calibration_mean and calibration_std were loaded correctly earlier in this cell)
    try:
        normalized_emg_part = (emg_part - calibration_mean) / calibration_std
    except NameError:
        print(f"ERROR: calibration_mean/std not available for demo processing (Class {gid}). Skipping.")
        continue # Skip to the next class if stats are missing

    # Calculate RMS on normalized EMG and raw IMU
    rms_emg = np.sqrt(np.mean(normalized_emg_part**2, axis=0))
    rms_imu = np.sqrt(np.mean(imu_part**2, axis=0))

    # Concatenate to get the final feature vector for the demo window
    rms_vec = np.concatenate([rms_emg, rms_imu])

    demo_rows.append((gid, name, rms_vec))

# assemble our demo set
y_true = [gid for gid,_,_ in demo_rows]
X_demo = np.vstack([vec for *_, vec in demo_rows])

# — 5) scale & predict —
X_scaled = scaler.transform(X_demo)
preds     = model.predict(X_scaled, verbose=0)
pred_ids  = preds.argmax(axis=1)

# — 6) report —
print("Demo results:")
for (true_id, true_name, _), pred in zip(demo_rows, pred_ids):
    print(f" • True: {true_id:2d} {true_name:17s} → Predicted: {pred:2d} {CLASSES[pred]}")
    
acc = np.mean(np.array(pred_ids) == np.array(y_true))
print(f"\nDemo accuracy (RMS‐window per class): {acc:.0%}")


Demo results:
 • True:  0 Rest              → Predicted:  0 Rest
 • True:  1 Wrist Flexion     → Predicted:  1 Wrist Flexion
 • True:  2 Wrist Extension   → Predicted:  2 Wrist Extension
 • True:  3 Elbow Flexion     → Predicted:  2 Wrist Extension
 • True:  4 Elbow Extension   → Predicted:  3 Elbow Flexion
 • True:  5 Hand Close        → Predicted:  5 Hand Close
 • True:  6 Hand Open         → Predicted:  6 Hand Open
 • True:  7 Forearm Pronation → Predicted:  7 Forearm Pronation
 • True:  8 Forearm Supination → Predicted:  8 Forearm Supination

Demo accuracy (RMS‐window per class): 78%
